In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pickle

import numpy as np
import pandas as pd

import json
from textblob import TextBlob

import nltk
import os
from scipy import spatial
import torch
import spacy
en_nlp = spacy.load('en')

In [ ]:
train = pd.read_json("/content/drive/MyDrive/Final-project/data/train-v1.1.json")

valid = pd.read_json("/content/drive/MyDrive/Final-project/data/dev-v1.1.json")

In [ ]:
train.shape, valid.shape

((442, 2), (48, 2))

In [ ]:
train.head(5)

,data,version
0,"{'title': 'University_of_Notre_Dame', 'paragra...",1.1
1,"{'title': 'Beyoncé', 'paragraphs': [{'context'...",1.1
2,"{'title': 'Montana', 'paragraphs': [{'context'...",1.1
3,"{'title': 'Genocide', 'paragraphs': [{'context...",1.1
4,"{'title': 'Antibiotics', 'paragraphs': [{'cont...",1.1


In [ ]:
train.iloc[1,0]['paragraphs'][0]

{'context': 'Beyoncé Giselle Knowles-Carter (/biːˈjɒnseɪ/ bee-YON-say) (born September 4, 1981) is an American singer, songwriter, record producer and actress. Born and raised in Houston, Texas, she performed in various singing and dancing competitions as a child, and rose to fame in the late 1990s as lead singer of R&B girl-group Destiny\'s Child. Managed by her father, Mathew Knowles, the group became one of the world\'s best-selling girl groups of all time. Their hiatus saw the release of Beyoncé\'s debut album, Dangerously in Love (2003), which established her as a solo artist worldwide, earned five Grammy Awards and featured the Billboard Hot 100 number-one singles "Crazy in Love" and "Baby Boy".',
 'qas': [{'answers': [{'answer_start': 269, 'text': 'in the late 1990s'}],
   'id': '56be85543aeaaa14008c9063',
   'question': 'When did Beyonce start becoming popular?'},
  {'answers': [{'answer_start': 207, 'text': 'singing and dancing'}],
   'id': '56be85543aeaaa14008c9065',
   'ques

In [ ]:
valid.iloc[1,0]['paragraphs'][0]

{'context': 'One of the most famous people born in Warsaw was Maria Skłodowska-Curie, who achieved international recognition for her research on radioactivity and was the first female recipient of the Nobel Prize. Famous musicians include Władysław Szpilman and Frédéric Chopin. Though Chopin was born in the village of Żelazowa Wola, about 60 km (37 mi) from Warsaw, he moved to the city with his family when he was seven months old. Casimir Pulaski, a Polish general and hero of the American Revolutionary War, was born here in 1745.',
 'qas': [{'answers': [{'answer_start': 188, 'text': 'Nobel Prize'},
    {'answer_start': 188, 'text': 'Nobel Prize'},
    {'answer_start': 188, 'text': 'Nobel Prize'}],
   'id': '5733a5f54776f41900660f45',
   'question': 'What was Maria Curie the first female recipient of?'},
  {'answers': [{'answer_start': 517, 'text': '1745'},
    {'answer_start': 517, 'text': '1745'},
    {'answer_start': 517, 'text': '1745'}],
   'id': '5733a5f54776f41900660f48',
   'que

In [ ]:
contexts = []
questions = []
answers_text = []
answers_start = []
for i in range(train.shape[0]):
    topic = train.iloc[i,0]['paragraphs']
    for sub_para in topic:
        for q_a in sub_para['qas']:
            questions.append(q_a['question'])
            answers_start.append(q_a['answers'][0]['answer_start'])
            answers_text.append(q_a['answers'][0]['text'])
            contexts.append(sub_para['context'])   

In [ ]:
df = pd.DataFrame({"context":contexts, "question": questions, "answer_start": answers_start, "text": answers_text})
df.shape

(87599, 4)

In [ ]:
df.to_csv("/content/drive/MyDrive/Final-project/data/csv/train.csv", index = None)

In [ ]:
df.head(5)

,context,question,answer_start,text
0,"Architecturally, the school has a Catholic cha...",To whom did the Virgin Mary allegedly appear i...,515,Saint Bernadette Soubirous
1,"Architecturally, the school has a Catholic cha...",What is in front of the Notre Dame Main Building?,188,a copper statue of Christ
2,"Architecturally, the school has a Catholic cha...",The Basilica of the Sacred heart at Notre Dame...,279,the Main Building
3,"Architecturally, the school has a Catholic cha...",What is the Grotto at Notre Dame?,381,a Marian place of prayer and reflection
4,"Architecturally, the school has a Catholic cha...",What sits on top of the Main Building at Notre...,92,a golden statue of the Virgin Mary


In [ ]:
# import os
# os.environ['KAGGLE_CONFIG_DIR'] = "/content/drive/MyDrive/Kaggle"

In [ ]:
%cd /content/drive/MyDrive/Kaggle
#Check the present working directory using pwd command

/content/drive/MyDrive/Kaggle


In [ ]:
# !kaggle datasets download -d dhznsdl/emb-model

In [ ]:
!ls

crawl-300d-2M.vec  infersent1.pkl  infersent2.pickle  kaggle.json  punkt


In [ ]:
# !unzip \*.zip  && rm *.zip

In [ ]:
# !kaggle datasets download -d alexanderliao/infersent

In [ ]:
# !kaggle datasets download -d dm4006/data-for-sent-emb-v2

In [ ]:
# !unzip \*.zip  && rm *.zip

In [ ]:
!ls

crawl-300d-2M.vec  infersent1.pkl  infersent2.pickle  kaggle.json  punkt


In [ ]:
paras = list(df["context"].drop_duplicates().reset_index(drop= True))

In [ ]:
len(paras)

18891

In [ ]:
type(paras), paras[10]

(list,
 "Father Joseph Carrier, C.S.C. was Director of the Science Museum and the Library and Professor of Chemistry and Physics until 1874. Carrier taught that scientific research and its promise for progress were not antagonistic to the ideals of intellectual and moral culture endorsed by the Church. One of Carrier's students was Father John Augustine Zahm (1851–1921) who was made Professor and Co-Director of the Science Department at age 23 and by 1900 was a nationally prominent scientist and naturalist. Zahm was active in the Catholic Summer School movement, which introduced Catholic laity to contemporary intellectual issues. His book Evolution and Dogma (1896) defended certain aspects of evolutionary theory as true, and argued, moreover, that even the great Church teachers Thomas Aquinas and Augustine taught something like it. The intervention of Irish American Catholics in Rome prevented Zahm's censure by the Vatican. In 1913, Zahm and former President Theodore Roosevelt embarked

In [ ]:
df.shape

(87599, 4)

In [ ]:
import nltk

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
blob = TextBlob(" ".join(paras))
sentences = [item.raw for item in blob.sentences]

In [ ]:
type(blob)

textblob.blob.TextBlob

In [ ]:
len(sentences)

92659

In [ ]:
type(sentences)

list

In [ ]:
# import numpy as np

In [ ]:
# npsentences = np.array(sentences)

In [ ]:
import sys
py_file_location = "/content/drive/MyDrive/Final-project/data"
sys.path.append(os.path.abspath(py_file_location))

In [ ]:
#model.py file from InferSent Repository

# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

"""
This file contains the definition of encoders used in https://arxiv.org/pdf/1705.02364.pdf
"""

import numpy as np
import time

import torch
import torch.nn as nn


class InferSent(nn.Module):

    def __init__(self, config):
        super(InferSent, self).__init__()
        self.bsize = config['bsize']
        self.word_emb_dim = config['word_emb_dim']
        self.enc_lstm_dim = config['enc_lstm_dim']
        self.pool_type = config['pool_type']
        self.dpout_model = config['dpout_model']
        self.version = 1 if 'version' not in config else config['version']

        self.enc_lstm = nn.LSTM(self.word_emb_dim, self.enc_lstm_dim, 1,
                                bidirectional=True, dropout=self.dpout_model)

        assert self.version in [1, 2]
        if self.version == 1:
            self.bos = '<s>'
            self.eos = '</s>'
            self.max_pad = True
            self.moses_tok = False
        elif self.version == 2:
            self.bos = '<p>'
            self.eos = '</p>'
            self.max_pad = False
            self.moses_tok = True

    def is_cuda(self):
        # either all weights are on cpu or they are on gpu
        return self.enc_lstm.bias_hh_l0.data.is_cuda

    def forward(self, sent_tuple):
        # sent_len: [max_len, ..., min_len] (bsize)
        # sent: (seqlen x bsize x worddim)
        sent, sent_len = sent_tuple

        # Sort by length (keep idx)
        sent_len_sorted, idx_sort = np.sort(sent_len)[::-1], np.argsort(-sent_len)
        sent_len_sorted = sent_len_sorted.copy()
        idx_unsort = np.argsort(idx_sort)

        idx_sort = torch.from_numpy(idx_sort).cuda() if self.is_cuda() \
            else torch.from_numpy(idx_sort)
        sent = sent.index_select(1, idx_sort)

        # Handling padding in Recurrent Networks
        sent_packed = nn.utils.rnn.pack_padded_sequence(sent, sent_len_sorted)
        sent_output = self.enc_lstm(sent_packed)[0]  # seqlen x batch x 2*nhid
        sent_output = nn.utils.rnn.pad_packed_sequence(sent_output)[0]

        # Un-sort by length
        idx_unsort = torch.from_numpy(idx_unsort).cuda() if self.is_cuda() \
            else torch.from_numpy(idx_unsort)
        sent_output = sent_output.index_select(1, idx_unsort)

        # Pooling
        if self.pool_type == "mean":
            sent_len = torch.FloatTensor(sent_len.copy()).unsqueeze(1).cuda()
            emb = torch.sum(sent_output, 0).squeeze(0)
            emb = emb / sent_len.expand_as(emb)
        elif self.pool_type == "max":
            if not self.max_pad:
                sent_output[sent_output == 0] = -1e9
            emb = torch.max(sent_output, 0)[0]
            if emb.ndimension() == 3:
                emb = emb.squeeze(0)
                assert emb.ndimension() == 2

        return emb

    def set_w2v_path(self, w2v_path):
        self.w2v_path = w2v_path

    def get_word_dict(self, sentences, tokenize=True):
        # create vocab of words
        word_dict = {}
        sentences = [s.split() if not tokenize else self.tokenize(s) for s in sentences]
        for sent in sentences:
            for word in sent:
                if word not in word_dict:
                    word_dict[word] = ''
        word_dict[self.bos] = ''
        word_dict[self.eos] = ''
        return word_dict

    def get_w2v(self, word_dict):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        # create word_vec with w2v vectors
        word_vec = {}
        with open(self.w2v_path, encoding='utf-8') as f:
            for line in f:
                word, vec = line.split(' ', 1)
                if word in word_dict:
                    word_vec[word] = np.fromstring(vec, sep=' ')
        print('Found %s(/%s) words with w2v vectors' % (len(word_vec), len(word_dict)))
        return word_vec

    def get_w2v_k(self, K):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        # create word_vec with k first w2v vectors
        k = 0
        word_vec = {}
        with open(self.w2v_path, encoding='utf-8') as f:
            for line in f:
                word, vec = line.split(' ', 1)
                if k <= K:
                    word_vec[word] = np.fromstring(vec, sep=' ')
                    k += 1
                if k > K:
                    if word in [self.bos, self.eos]:
                        word_vec[word] = np.fromstring(vec, sep=' ')

                if k > K and all([w in word_vec for w in [self.bos, self.eos]]):
                    break
        return word_vec

    def build_vocab(self, sentences, tokenize=True):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        word_dict = self.get_word_dict(sentences, tokenize)
        self.word_vec = self.get_w2v(word_dict)
        print('Vocab size : %s' % (len(self.word_vec)))

    # build w2v vocab with k most frequent words
    def build_vocab_k_words(self, K):
        assert hasattr(self, 'w2v_path'), 'w2v path not set'
        self.word_vec = self.get_w2v_k(K)
        print('Vocab size : %s' % (K))

    def update_vocab(self, sentences, tokenize=True):
        assert hasattr(self, 'w2v_path'), 'warning : w2v path not set'
        assert hasattr(self, 'word_vec'), 'build_vocab before updating it'
        word_dict = self.get_word_dict(sentences, tokenize)

        # keep only new words
        for word in self.word_vec:
            if word in word_dict:
                del word_dict[word]

        # udpate vocabulary
        if word_dict:
            new_word_vec = self.get_w2v(word_dict)
            self.word_vec.update(new_word_vec)
        else:
            new_word_vec = []
        print('New vocab size : %s (added %s words)'% (len(self.word_vec), len(new_word_vec)))

    def get_batch(self, batch):
        # sent in batch in decreasing order of lengths
        # batch: (bsize, max_len, word_dim)
        embed = np.zeros((len(batch[0]), len(batch), self.word_emb_dim))

        for i in range(len(batch)):
            for j in range(len(batch[i])):
                embed[j, i, :] = self.word_vec[batch[i][j]]

        return torch.FloatTensor(embed)

    def tokenize(self, s):
        from nltk.tokenize import word_tokenize
        if self.moses_tok:
            s = ' '.join(word_tokenize(s))
            s = s.replace(" n't ", "n 't ")  # HACK to get ~MOSES tokenization
            return s.split()
        else:
            return word_tokenize(s)

    def prepare_samples(self, sentences, bsize, tokenize, verbose):
        sentences = [[self.bos] + s.split() + [self.eos] if not tokenize else
                     [self.bos] + self.tokenize(s) + [self.eos] for s in sentences]
        n_w = np.sum([len(x) for x in sentences])

        # filters words without w2v vectors
        for i in range(len(sentences)):
            s_f = [word for word in sentences[i] if word in self.word_vec]
            if not s_f:
                import warnings
                warnings.warn('No words in "%s" (idx=%s) have w2v vectors. \
                               Replacing by "</s>"..' % (sentences[i], i))
                s_f = [self.eos]
            sentences[i] = s_f

        lengths = np.array([len(s) for s in sentences])
        n_wk = np.sum(lengths)
        if verbose:
            print('Nb words kept : %s/%s (%.1f%s)' % (
                        n_wk, n_w, 100.0 * n_wk / n_w, '%'))

        # sort by decreasing length
        lengths, idx_sort = np.sort(lengths)[::-1], np.argsort(-lengths)
        sentences = np.array(sentences)[idx_sort]

        return sentences, lengths, idx_sort

    def encode(self, sentences, bsize=64, tokenize=True, verbose=False):
        tic = time.time()
        sentences, lengths, idx_sort = self.prepare_samples(
                        sentences, bsize, tokenize, verbose)

        embeddings = []
        for stidx in range(0, len(sentences), bsize):
            batch = self.get_batch(sentences[stidx:stidx + bsize])
            if self.is_cuda():
                batch = batch.cuda()
            with torch.no_grad():
                batch = self.forward((batch, lengths[stidx:stidx + bsize])).data.cpu().numpy()
            embeddings.append(batch)
        embeddings = np.vstack(embeddings)

        # unsort
        idx_unsort = np.argsort(idx_sort)
        embeddings = embeddings[idx_unsort]

        if verbose:
            print('Speed : %.1f sentences/s (%s mode, bsize=%s)' % (
                    len(embeddings)/(time.time()-tic),
                    'gpu' if self.is_cuda() else 'cpu', bsize))
        return embeddings

    def visualize(self, sent, tokenize=True):

        sent = sent.split() if not tokenize else self.tokenize(sent)
        sent = [[self.bos] + [word for word in sent if word in self.word_vec] + [self.eos]]

        if ' '.join(sent[0]) == '%s %s' % (self.bos, self.eos):
            import warnings
            warnings.warn('No words in "%s" have w2v vectors. Replacing \
                           by "%s %s"..' % (sent, self.bos, self.eos))
        batch = self.get_batch(sent)

        if self.is_cuda():
            batch = batch.cuda()
        output = self.enc_lstm(batch)[0]
        output, idxs = torch.max(output, 0)
        # output, idxs = output.squeeze(), idxs.squeeze()
        idxs = idxs.data.cpu().numpy()
        argmaxs = [np.sum((idxs == k)) for k in range(len(sent[0]))]

        # visualize model
        import matplotlib.pyplot as plt
        x = range(len(sent[0]))
        y = [100.0 * n / np.sum(argmaxs) for n in argmaxs]
        plt.xticks(x, sent[0], rotation=45)
        plt.bar(x, y)
        plt.ylabel('%')
        plt.title('Visualisation of words importance')
        plt.show()

        return output, idxs


In [ ]:
# MODEL_PATH = '/content/drive/MyDrive/Kaggle/infersent1.pkl'
# params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
#                 'pool_type': 'max', 'dpout_model': 0.0}
# infersent = InferSent(params_model)
# infersent.load_state_dict(torch.load(MODEL_PATH))

In [ ]:
MODEL_PATH = '/content/drive/MyDrive/Kaggle/infersent2.pickle'
params_model = {'bsize': 64, 'word_emb_dim': 300, 'enc_lstm_dim': 2048,
                'pool_type': 'max', 'dpout_model': 0.0}
infersent2 = InferSent(params_model)
infersent2.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [ ]:
# !kaggle datasets download -d yekenot/fasttext-crawl-300d-2m

In [ ]:
# !unzip \*.zip  && rm *.zip

In [ ]:
!ls

crawl-300d-2M.vec  infersent1.pkl  infersent2.pickle  kaggle.json  punkt


In [ ]:
# Glove_PATH = '/content/drive/MyDrive/Kaggle/glove.840B.300d.txt'
# infersent.set_w2v_path(Glove_PATH)

In [ ]:
VEC_PATH = '/content/drive/MyDrive/Kaggle/crawl-300d-2M.vec'
infersent2.set_w2v_path(VEC_PATH)

In [ ]:
# infersent.build_vocab(sentences, tokenize=True)

In [ ]:
infersent2.build_vocab(sentences,tokenize=True) #This worked better

Found 89218(/109718) words with w2v vectors
Vocab size : 89218


In [ ]:
for i in range (0,10):
  print(i,sentences[i])

print(type(sentences))

0 Architecturally, the school has a Catholic character.
1 Atop the Main Building's gold dome is a golden statue of the Virgin Mary.
2 Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes".
3 Next to the Main Building is the Basilica of the Sacred Heart.
4 Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection.
5 It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858.
6 At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.
7 As at most other universities, Notre Dame's students run a number of news media outlets.
8 The nine student-run outlets include three newspapers, both a radio and television station, and several magazines and journals.
9 Begun as a one-page journal in September 1876, the Scholastic magaz

In [ ]:
print(len(sentences))

92659


In [ ]:
dict_embeddings = {}

In [ ]:
for i in range(len(sentences)):
# for i in range (0,20):
    print(i)
    dict_embeddings[sentences[i]] = infersent2.encode([sentences[i]],tokenize=True, verbose=True)

Streaming output truncated to the last 5000 lines.
34239
Nb words kept : 41/44 (93.2%)
Speed : 2.6 sentences/s (cpu mode, bsize=64)
34240
Nb words kept : 56/59 (94.9%)
Speed : 2.0 sentences/s (cpu mode, bsize=64)
34241
Nb words kept : 36/38 (94.7%)
Speed : 3.0 sentences/s (cpu mode, bsize=64)
34242
Nb words kept : 23/27 (85.2%)
Speed : 4.7 sentences/s (cpu mode, bsize=64)
34243
Nb words kept : 10/13 (76.9%)
Speed : 10.2 sentences/s (cpu mode, bsize=64)
34244
Nb words kept : 27/31 (87.1%)
Speed : 4.1 sentences/s (cpu mode, bsize=64)
34245
Nb words kept : 67/71 (94.4%)
Speed : 1.7 sentences/s (cpu mode, bsize=64)
34246
Nb words kept : 78/80 (97.5%)
Speed : 1.4 sentences/s (cpu mode, bsize=64)
34247
Nb words kept : 31/33 (93.9%)
Speed : 3.2 sentences/s (cpu mode, bsize=64)
34248
Nb words kept : 16/18 (88.9%)
Speed : 6.1 sentences/s (cpu mode, bsize=64)
34249
Nb words kept : 24/26 (92.3%)
Speed : 3.9 sentences/s (cpu mode, bsize=64)
34250
Nb words kept : 25/28 (89.3%)
Speed : 4.1 sentences

In [ ]:
with open('/content/drive/MyDrive/Final-project/data/dict_sentences92659.pickle', 'wb') as handle:
    pickle.dump(dict_embeddings, handle)

In [ ]:
with open("/content/drive/MyDrive/data/weights/dict_sentences92659.pickle", "rb") as f:
    dict_embeddings = pickle.load(f)

In [ ]:
questions = list(df["question"])

In [ ]:
len(questions)

87599

In [ ]:
for i in range (0,10):
  print(i,questions[i])

print(type(questions))

0 To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?
1 What is in front of the Notre Dame Main Building?
2 The Basilica of the Sacred heart at Notre Dame is beside to which structure?
3 What is the Grotto at Notre Dame?
4 What sits on top of the Main Building at Notre Dame?
5 When did the Scholastic Magazine of Notre dame begin publishing?
6 How often is Notre Dame's the Juggler published?
7 What is the daily student paper at Notre Dame called?
8 How many student news papers are found at Notre Dame?
9 In what year did the student paper Common Sense begin publication at Notre Dame?
<class 'list'>


In [ ]:
for i in range(len(questions)):
# for i in range (0,20):
    print(i)
    dict_embeddings[questions[i]] = infersent2.encode([questions[i]], tokenize=True, verbose=True)
# dict_embeddings[questions[i]] = embeddings2 = infersent2.encode(questions, tokenize=True)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [ ]:
with open('/content/drive/MyDrive/Final-project/data/dict_questions87599.pickle', 'wb') as handle:
    pickle.dump(dict_embeddings, handle)

In [ ]:
dict_embeddings["Its main seminary, Moreau Seminary, is located on the campus across St. Joseph lake from the Main Building."][0]

array([-0.05209418,  0.07462937,  0.05871293, ...,  0.08557521,
        0.00767955, -0.00119523], dtype=float32)

In [ ]:
d1 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 0}
d2 = {key:dict_embeddings[key] for i, key in enumerate(dict_embeddings) if i % 2 == 1}

In [ ]:
d1,d2

({'Architecturally, the school has a Catholic character.': array([[-0.01541214, -0.05413497,  0.01449185, ...,  0.0406326 ,
           0.01274821, -0.01476117]], dtype=float32),
  'At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.': array([[-0.02252096, -0.01537143,  0.1414491 , ...,  0.00989005,
          -0.00461595,  0.06525892]], dtype=float32),
  'How many departments are within the Stinson-Remick Hall of Engineering?': array([[-0.01380435, -0.07716699,  0.02579542, ..., -0.00062815,
          -0.00991026, -0.02370446]], dtype=float32),
  'How many student news papers are found at Notre Dame?': array([[-0.01380435, -0.03168311,  0.03808569, ...,  0.00572285,
           0.03413418, -0.0264217 ]], dtype=float32),
  "How often is Notre Dame's the Juggler published?": array([[-0.01380435, -0.04147759,  0.11102663, ..., -0.01407725,
           0.04923555, -0.00183876]], dtype=float32),
  'Imm

In [ ]:
# temp = dict(d1)
# temp.update(d2)

In [ ]:
# len(temp)

40

In [ ]:
with open('/content/drive/MyDrive/Final-project/data/dict_embeddings1.pickle', 'wb') as handle:
    pickle.dump(d1, handle)

In [ ]:
with open('/content/drive/MyDrive/Final-project/data/dict_embeddings2.pickle', 'wb') as handle:
    pickle.dump(d2, handle)